# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [3]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [5]:
# set up environment
# Load environment variables in a file called .env
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [6]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()
response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[ {"role": "user", "content": question}],
    )
result = response.choices[0].message.content
display(Markdown(result))

The code snippet you've provided uses Python's `yield from` syntax, alongside a set comprehension, to yield each author from a collection of books. Let's break it down step by step:

1. **Set Comprehension**: The component `{book.get("author") for book in books if book.get("author")}` is a set comprehension. This means it's creating a set (a collection of unique elements) containing the authors of the books. 

    - `for book in books`: This part iterates through a collection named `books`, which is expected to be a list or any iterable containing book records (likely dictionaries).
    - `book.get("author")`: This retrieves the value associated with the key `"author"` from each `book`. The `get` method is used so that if `"author"` isn't present, it returns `None` instead of raising a KeyError.
    - `if book.get("author")`: This condition filters the comprehension to only include entries where the `"author"` key exists and has a truthy value (i.e., it's not `None`, `False`, or an empty string).

2. **Yield from**: The `yield from` syntax is used to delegate generator behavior to another generator (or in this case, an iterable). When used here, it allows the code to yield each unique author from the set generated by the comprehension one at a time.

Combining these pieces, the entire line of code effectively does the following:

- It constructs a set of unique authors from the `books` collection, filtering out any entries where the author is not specified.
- It then yields each of the authors one by one, allowing for iteration over them in a generator-friendly manner.

### Why Use This Approach?
- **Uniqueness**: By using a set comprehension, you ensure that each author is yielded only once, even if multiple books have the same author.
- **Efficiency**: Using `yield from` provides a way to yield each item lazily, meaning it doesn’t compute and store all authors in memory upfront, leading to lower memory usage and potentially better performance for large datasets.
- **Clean Code**: This approach is quite readable and concise, leveraging Python's features for clarity.

### Example Usage
If you had a list of dictionaries like this:

```python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": "Author A"}, # Duplicate author
    {"title": "Book Four"}, # No author
]
```

Using the provided line of code would yield `Author A` and `Author B`, but only once for each author.

In [8]:
# Get Llama 3.2 to answer

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[ {"role": "user", "content": question}]
)

result = response.choices[0].message.content
display(Markdown(result))


This piece of code is written in Python and utilizes the `yield from` keyword, which was introduced in Python 3.3 as part of its generator support.

Let's break down what this code does:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. `{...for book in books if book.get("author")}`: This is a dictionary comprehension, which creates a new dictionary (in this case) from an iterable.

2. `if book.get("author")`: It only includes entries where the `"author"` key exists.

3. `yield from {...}`: The `yield from` keyword tells Python that you want to delegarte execution to other iterable(s). In this case, it yields all values returned by `{...}` in "round-robin" fashion

So, putting it together, this code takes a list of dictionaries (`books`) and uses dictionary comprehension to filter out books without an `"author"` key. Then, within the filtered list's `__iter__()`, it iterates over each entry from this comprehension which returns author names, yielding each resulting string into an output iterable (usually some sort of stack).

In essence, this line can be considered as implementing "lazy loading" for a list of authors:

```python
for author in yield:
    print(author)
```

Will cause it to load all the books only when `author` is invoked.

Here's how you might use it (please let me know what your use case is): 

``` 
books = [
   {'title': 'Alice'},
   {'book_id': 1, 'author': 'Bob'},
   {'title': 'Charlie'}
]

for author in yield from {book.get("author") for book in books if book.get("author")}: 
    print(author)
```

Output for this example:

```plain
Bob
```

Here's an example code snippet demonstrating how you might use it to load authors on demand:

```python
# Define some class methods that will handle author retrieval (suggestion for real-world application).
def get_all(authors):
    """
    Loads all available authors
   """
    # Return the authors list from your data source
    return [
        {'id': 1, 'name': 'Author1'},
        {"id":2,'name':'Author2'}
]
    
def by_id(loadAuthorsFunc,name):
  
  for book in books:
    if book.get('author') == name:
      print(book)
      # Assuming that the returned value is a tuple
      return loadAuthorsFunc(author:book['author'])
loadAuthors = get_all
allAuthor_names = ['Author1','Author2']
      
# We are iterating over each name. 
for authorName in allAuthor_names:
   result = loadAuthors()

```

Note, `get_author()` yields the name if you wanted to implement it similar way to `by_id()`